# Setup

In [102]:
# install.packages("dplyr")
# install.packages("pROC")
# install.packages("ggplot2")

In [103]:
library(dplyr)
library(pROC)
library(ggplot2)

In [104]:
input_dir_1 <- "SimLRs/TRIO/familias-cleaned-LR"
output_dir_1 <- "SimLRs/TRIO/trio_ROC"

# Find all Trio cleaned LR files

In [105]:
clean_files <- list.files(
  path       = input_dir_1,
  pattern    = "^Trio_.*_cleanLR\\.txt$",
  full.names = TRUE
)

if (length(clean_files) == 0) {
  stop("No Trio_*.txt cleaned files found.")
}

cat("Found", length(clean_files), "Trio cleaned LR files:\n")
print(basename(clean_files))

Found 7 Trio cleaned LR files:
[1] "Trio_23aSTR_cleanLR.txt"         "Trio_23astr-90iisnp_cleanLR.txt"
[3] "Trio_23astr-94iisnp_cleanLR.txt" "Trio_27aSTR_cleanLR.txt"        
[5] "Trio_27astr-94iisnp_cleanLR.txt" "Trio_90iisnp_cleanLR.txt"       
[7] "Trio_94iisnp_cleanLR.txt"       


# Create function to compute ROC +AUC + ggplot ROC for one Trio panel

In [106]:
compute_trio_roc <- function(file_path, output_dir_1) {
    df <- read.table(file_path, header = TRUE)

    #Get name of marker panel
    base <- basename(file_path)
    panel_name <- sub("^Trio_(.*?)_cleanLR\\.txt$", "\\1", base) 
    
    #Get colums labels of dataframe
    col_true <- names(df)[1]
    col_unrel <- names(df)[2]

    # Convert to log10(LR)
    LR_true   <- log10(df[[col_true]])
    LR_unrel  <- log10(df[[col_unrel]])

    # Labels: 1 = Trio, 0 = unrelated
    y   <- c(rep(1, length(LR_true)), rep(0, length(LR_unrel)))
    scores <- c(LR_true, LR_unrel)

    # ROC via pROC
    roc_obj <- roc(response = y, predictor = scores)
    auc_val <- as.numeric(auc(roc_obj))

    # Build data frame for ggplot ROC curve
    roc_df <- data.frame(
    FPR       = 1 - roc_obj$specificities,
    TPR       = roc_obj$sensitivities,
    threshold = roc_obj$thresholds,
    Panel     = panel_name
    )

    # Individual ggplot ROC curve
    p_individual <- ggplot(roc_df, aes(x = FPR, y = TPR)) +
    geom_line(size = 1.1, color = "steelblue") +
    geom_abline(slope = 1, intercept = 0, linetype = "dashed", color = "grey60") +
    coord_equal() +
    labs(
      title    = paste("ROC Curve - Trio (", panel_name, ")", sep = ""),
      subtitle = paste("AUC:", format(auc_val, digits = 15, scientific = TRUE)),
      x        = "False Positive Rate (1 - Specificity)",
      y        = "True Positive Rate (Sensitivity)"
    ) +
    theme_minimal(base_size = 14)

    # Save per-panel ROC figure
    roc_png <- file.path(output_dir_1, paste0("ROC_Trio_", panel_name, ".png"))
    ggsave(roc_png, plot = p_individual, width = 7, height = 6, dpi = 300)
    cat("Saved individual ROC plot:", roc_png, "\n")

    # Return:
    # - summary row for AUC table
    # - roc_df for combined plotting
    list(
        auc_row = data.frame(
        Panel = panel_name,
        AUC = format(auc_val, digits = 20, scientific = TRUE),
        N_sim = nrow(df),
        stringsAsFactors = FALSE
        ),
        roc_df = roc_df
    )
}

# Loop over all Trio panels

In [107]:
auc_list   <- vector("list", length(clean_files))
rocdf_list <- vector("list", length(clean_files))

for (i in seq_along(clean_files)) {
  f <- clean_files[i]
  cat("\nProcessing:", basename(f), "\n")
  res <- compute_trio_roc(f, output_dir_1)
  auc_list[[i]]   <- res$auc_row
  rocdf_list[[i]] <- res$roc_df
}

auc_table <- bind_rows(auc_list)
roc_all   <- bind_rows(rocdf_list)


Processing: Trio_23aSTR_cleanLR.txt 


Setting levels: control = 0, case = 1

Setting direction: controls < cases



Saved individual ROC plot: SimLRs/TRIO/trio_ROC/ROC_Trio_23aSTR.png 

Processing: Trio_23astr-90iisnp_cleanLR.txt 


Setting levels: control = 0, case = 1

Setting direction: controls < cases



Saved individual ROC plot: SimLRs/TRIO/trio_ROC/ROC_Trio_23astr-90iisnp.png 

Processing: Trio_23astr-94iisnp_cleanLR.txt 


...

Saved individual ROC plot: SimLRs/TRIO/trio_ROC/ROC_Trio_27astr-94iisnp.png 

Processing: Trio_90iisnp_cleanLR.txt 


Setting levels: control = 0, case = 1

Setting direction: controls < cases



Saved individual ROC plot: SimLRs/TRIO/trio_ROC/ROC_Trio_90iisnp.png 

Processing: Trio_94iisnp_cleanLR.txt 


Setting levels: control = 0, case = 1

Setting direction: controls < cases



Saved individual ROC plot: SimLRs/TRIO/trio_ROC/ROC_Trio_94iisnp.png 


# Save AUC summary table

In [108]:
auc_outfile <- file.path(output_dir_1, "AUC_summary_Trio_panels.txt")

write.table(
  auc_table,
  file      = auc_outfile,
  sep       = "\t",
  quote     = FALSE,
  row.names = FALSE
)

cat("\nAUC summary table saved to:", auc_outfile, "\n")
print(auc_table)


AUC summary table saved to: SimLRs/TRIO/trio_ROC/AUC_summary_Trio_panels.txt 
           Panel   AUC N_sim
1         23aSTR 1e+00 10000
2 23astr-90iisnp 1e+00 10000
3 23astr-94iisnp 1e+00 10000
4         27aSTR 1e+00 10000
5 27astr-94iisnp 1e+00 10000
6        90iisnp 1e+00 10000
7        94iisnp 1e+00 10000


# Combined ROC plot (all panels together)

In [109]:

# Create nice labels including AUC in legend
auc_labels <- auc_table %>%
  mutate(
    PanelLabel = paste0(
      Panel,
      " (AUC=",
      format(as.numeric(AUC), digits = 6, scientific = TRUE),
      ")"
    )
  )

# Map raw Panel names to PanelLabel for legend
panel_label_map <- setNames(auc_labels$PanelLabel, auc_labels$Panel)

roc_all$Panel <- factor(roc_all$Panel, levels = auc_labels$Panel)

p_combined <- ggplot(roc_all, aes(x = FPR, y = TPR, color = Panel)) +
  geom_line(size = 1.1) +
  geom_abline(slope = 1, intercept = 0, linetype = "dashed", color = "grey60") +
  coord_equal() +
  scale_color_discrete(labels = panel_label_map[levels(roc_all$Panel)]) +
  labs(
    title = "ROC Curves - Trio, all marker panels",
    x     = "False Positive Rate (1 - Specificity)",
    y     = "True Positive Rate (Sensitivity)",
    color = "Panel (AUC)"
  ) +
  theme_minimal(base_size = 14)

combined_png <- file.path(output_dir_1, "ROC_Trio_all_panels.png")
ggsave(combined_png, plot = p_combined, width = 8, height = 7, dpi = 300)

cat("\nCombined ROC plot saved to:", combined_png, "\n")


Combined ROC plot saved to: SimLRs/TRIO/trio_ROC/ROC_Trio_all_panels.png 
